In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import os
import pandas as pd
import seaborn as sb
import statistics as st

from Bio import SeqIO

sb.set()
pd.set_option("display.max_rows", None)

# Evaluation of Swarm on Callahan data

The following notebook describes the steps and results of the evaluation.

In [ ]:
# Initial files and directories:
#
# model_supported_callahan
# |- data
# |  |- balanced
# |  |  \- BalancedRefSeqs.fasta  # provided reference sequences of 'balanced' data set
# |  |
# |  \- hmp
# |     \- HMP_MOCK.fasta  # provided reference sequences of 'hmp' data set
# |
# |- evaluation # will contain the evaluation plots and tables
# |
# |- outputs # will contain the cluster and metric outputs
# |
# \- tasks  # task files for the different runs of Swarm

The provided reference sequences are part of the [Supplementary Software](https://static-content.springer.com/esm/art%3A10.1038%2Fnmeth.3869/MediaObjects/41592_2016_BFnmeth3869_MOESM270_ESM.zip) of the DADA2 paper.   

## Analysis workflow

The data sets are preprocessed as described in Callahan et al., *DADA2: High-resolution sample inference from Illumina amplicon data* (https://doi.org/10.1038/nmeth.3869),
except that the minimum sequence abundance is set to 1 (not 2) for the sake of a fair comparison with other data sets and tools.

The taxonomic assignment is obtained by merging (using `USEARCH -fastq_mergepairs`) and dereplicating (using `USEARCH -derep_fulllength`) the reads, 
and matching them with the respective reference sequences (using `VSEARCH --usearch_global`).

Swarm (v3) is executed with and without the fastidious refinement step.
Runs without refinement have the suffix `__nf`, while those with refinement end with `__2f`.

## Commands

The following commands prepare and cluster the data sets. The results are evaluated below.

In order to execute the workflow as provided here, the `tools` subdirectory of the overall repository should contain the USEARCH binaries `usearch8.0.1623_i86linux32` and `usearch10.0.240_i86linux32`, but the paths can be adjusted.
In addition, VSEARCH is expected to be accessible through the `vsearch` command.  

IMPORTANT: The commands are not intended to be executed from this notebook. They should be executed from the root directory of the overall repository.

In [ ]:
%%bash

TOOLS_DIR=tools
ANALYSIS_DIR=analyses/model_supported_callahan
DATA_DIR=${ANALYSIS_DIR}/data
OUTPUT_DIR=${ANALYSIS_DIR}/outputs

USEARCH8_PATH=${TOOLS_DIR}/usearch8.0.1623_i86linux32 # adjust to your system
USEARCH10_PATH=${TOOLS_DIR}/usearch10.0.240_i86linux32 # adjust to your system

SWARM=${TOOLS_DIR}/swarm-3.0.0/bin/swarm # adjust to your system

RUNS=( swarm_v3__nf swarm_v3__2f )

## balanced
python -m scripts.analyses.analysis_callahan reference balanced ${DATA_DIR}/balanced/BalancedRefSeqs.fasta ${DATA_DIR}/balanced/callahan.fasta 

# paired
python -m scripts.analyses.analysis_callahan prepare balanced ${DATA_DIR}/balanced --min_size 1 --usearch8 ${USEARCH8_PATH} --usearch10 ${USEARCH10_PATH}
python -m scripts.analyses.analysis_callahan taxonomy ${DATA_DIR}/balanced/ERR777695_pfmd.fastq ${DATA_DIR}/balanced/callahan.fasta ${DATA_DIR}/balanced/bp_callahan_0.97.tax 0.97

READS=bp:${DATA_DIR}/balanced/ERR777695_pfmd.fastq
TAX=callahan:${DATA_DIR}/balanced/bp_callahan_0.97.tax
for R in "${RUNS[@]}"; do
    python -m scripts.analyses.analysis_callahan run_swarm ${R} ${READS} ${ANALYSIS_DIR}/tasks/${R}.txt ${OUTPUT_DIR}/balanced_paired/${R} --tax_files ${TAX} --swarm ${SWARM}
    for F in ${OUTPUT_DIR}/balanced_paired/${R}/*__metrics.csv; do mv ${F} ${OUTPUT_DIR}/balanced_paired/${R}/${R}_${F##*/}; done
done


# single
python -m scripts.analyses.analysis_callahan prepare balanced ${DATA_DIR}/balanced --single --min_size 1 --usearch8 ${USEARCH8_PATH} --usearch10 ${USEARCH10_PATH}
python -m scripts.analyses.analysis_callahan taxonomy ${DATA_DIR}/balanced/ERR777695_1_sfd.fastq ${DATA_DIR}/balanced/callahan.fasta ${DATA_DIR}/balanced/bs_callahan_0.97.tax 0.97

READS=bs:${DATA_DIR}/balanced/ERR777695_1_sfd.fastq
TAX=callahan:${DATA_DIR}/balanced/bs_callahan_0.97.tax
for R in "${RUNS[@]}"; do
    python -m scripts.analyses.analysis_callahan run_swarm ${R} ${READS} ${ANALYSIS_DIR}/tasks/${R}.txt ${OUTPUT_DIR}/balanced_single/${R} --tax_files ${TAX} --swarm ${SWARM}
    for F in ${OUTPUT_DIR}/balanced_single/${R}/*__metrics.csv; do mv ${F} ${OUTPUT_DIR}/balanced_single/${R}/${R}_${F##*/}; done
done


## hmp
python -m scripts.analyses.analysis_callahan reference hmp ${DATA_DIR}/hmp/HMP_MOCK.fasta ${DATA_DIR}/hmp/callahan.fasta 

# paired
python -m scripts.analyses.analysis_callahan prepare hmp ${DATA_DIR}/hmp --min_size 1 --usearch8 ${USEARCH8_PATH} --usearch10 ${USEARCH10_PATH}
python -m scripts.analyses.analysis_callahan taxonomy ${DATA_DIR}/hmp/Mock1_S1_L001_pfmd.fastq ${DATA_DIR}/hmp/callahan.fasta ${DATA_DIR}/hmp/hp_callahan_0.97.tax 0.97

READS=hp:${DATA_DIR}/hmp/Mock1_S1_L001_pfmd.fastq
TAX=callahan:${DATA_DIR}/hmp/hp_callahan_0.97.tax
for R in "${RUNS[@]}"; do
    python -m scripts.analyses.analysis_callahan run_swarm ${R} ${READS} ${ANALYSIS_DIR}/tasks/${R}.txt ${OUTPUT_DIR}/hmp_paired/${R} --tax_files ${TAX} --swarm ${SWARM}
    for F in ${OUTPUT_DIR}/hmp_paired/${R}/*__metrics.csv; do mv ${F} ${OUTPUT_DIR}/hmp_paired/${R}/${R}_${F##*/}; done
done


# single
python -m scripts.analyses.analysis_callahan prepare hmp ${DATA_DIR}/hmp --single --min_size 1 --usearch8 ${USEARCH8_PATH} --usearch10 ${USEARCH10_PATH}
python -m scripts.analyses.analysis_callahan taxonomy ${DATA_DIR}/hmp/Mock1_S1_L001_R1_001_sfd.fastq ${DATA_DIR}/hmp/callahan.fasta ${DATA_DIR}/hmp/hs_callahan_0.97.tax 0.97

READS=hs:${DATA_DIR}/hmp/Mock1_S1_L001_R1_001_sfd.fastq
TAX=callahan:${DATA_DIR}/hmp/hs_callahan_0.97.tax
for R in "${RUNS[@]}"; do
    python -m scripts.analyses.analysis_callahan run_swarm ${R} ${READS} ${ANALYSIS_DIR}/tasks/${R}.txt ${OUTPUT_DIR}/hmp_single/${R} --tax_files ${TAX} --swarm ${SWARM}
    for F in ${OUTPUT_DIR}/hmp_single/${R}/*__metrics.csv; do mv ${F} ${OUTPUT_DIR}/hmp_single/${R}/${R}_${F##*/}; done
done

## Evaluation

**Configuration**

In [2]:
data_sets = ['balanced', 'hmp']
read_types = ['single', 'paired']
ground_truths = ['callahan']

opts = ['swarm_v3__nf', 'swarm_v3__2f']

read_files = {'balanced_single': 'ERR777695_1_sfd.fastq', 'balanced_paired': 'ERR777695_pfmd.fastq',
              'hmp_single': 'Mock1_S1_L001_R1_001_sfd.fastq', 'hmp_paired': 'Mock1_S1_L001_pfmd.fastq'}

data_dir = 'data'
results_dir = 'outputs'
eval_dir = 'evaluation'

### Number of clusters and amplicons

Reads the input files and the cluster outputs for all data sets and compares the number of clusters and amplicons.

In [3]:
# Requires the input and OTU files. Alternatively, the evaluation can use the stored information (see below).
df_columns = ['data_set', 'tool', 'mode', 'refinement', 'threshold', 'num_input_amplicons', 'input_mass', 'num_clusters', 'num_output_amplicons', 'output_mass', 'ds', 'rt']

rows = []

for rt in read_types:
    for ds in data_sets:
        run_name = '%s_%s' % (ds, rt)
        
        seq_file = '%s/%s/%s' % (data_dir, ds, read_files[run_name]) # the input sequences
        num_input_amplicons = 0
        input_mass = 0
        with open(seq_file, 'r') as in_file:
            for record in SeqIO.parse(in_file, 'fastq'):
                num_input_amplicons += 1
                input_mass += int(record.id.split('_')[-1])
        
        for opt in opts:
            otu_files = [f for f in os.listdir('%s/%s/%s/' % (results_dir, run_name, opt)) if f.endswith('_otus.txt')]

            for f in otu_files:
                otu_file = '%s/%s/%s/%s' % (results_dir, run_name, opt, f)
                                        
                num_output_amplicons = 0
                num_clusters = 0
                output_mass = 0
                with open(otu_file, 'r') as in_file:
                    for line in in_file:
                        num_output_amplicons += len(line.strip().split(' '))
                        num_clusters += 1
                        output_mass += sum([int(m.split('_')[-1]) for m in line.strip().split(' ')])
                        
                tool = 'swarm'
                mode = 'swarm'
                refinement = opt.split('__')[1]
                threshold = float(f.split('_')[-2])

                rows.append([run_name, tool, mode, refinement, threshold, num_input_amplicons, input_mass, num_clusters, num_output_amplicons, output_mass, ds, rt])
            
df_counts = pd.DataFrame(rows, columns = df_columns)
df_counts.sort_values(by = ['data_set', 'tool', 'mode', 'refinement', 'threshold'], inplace = True)

*Column descriptions:*   
`num_input_amplicons`: The number of entries in the corresponding input file.   
`input_mass`: The sum of the abundances of all entries in the input file.   
`num_clusters`: The number of computed clusters.   
`num_output_amplicons`: The number of amplicons contained in the clusters.   
`output_mass`: The sum of the abundances of all amplicons contained in the clusters.   

In [4]:
df_counts[['data_set', 'tool', 'refinement', 'threshold', 'num_input_amplicons', 'input_mass', 'num_clusters', 'num_output_amplicons', 'output_mass']]

,data_set,tool,refinement,threshold,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass
32,balanced_paired,swarm,2f,1.0,21808,467652,2866,21808,467652
22,balanced_paired,swarm,nf,1.0,21808,467652,3684,21808,467652
24,balanced_paired,swarm,nf,2.0,21808,467652,2383,21808,467652
25,balanced_paired,swarm,nf,3.0,21808,467652,1747,21808,467652
26,balanced_paired,swarm,nf,4.0,21808,467652,1403,21808,467652
27,balanced_paired,swarm,nf,5.0,21808,467652,1217,21808,467652
28,balanced_paired,swarm,nf,6.0,21808,467652,1037,21808,467652
29,balanced_paired,swarm,nf,7.0,21808,467652,876,21808,467652
30,balanced_paired,swarm,nf,8.0,21808,467652,728,21808,467652
31,balanced_paired,swarm,nf,9.0,21808,467652,605,21808,467652


In [5]:
df_counts.to_csv('%s/df_counts.csv' % eval_dir, sep = ';', index = False)
#df_counts = pd.read_csv('%s/df_counts.csv' % eval_dir, sep = ';')

### Clustering quality

In [6]:
# Requires the metrics files. Alternatively, the evaluation can use the stored information (see below).
dfs = []
for ds in data_sets:
    for rt in read_types:
        run_name = '%s_%s' % (ds, rt)

        for opt in opts:
            for gt in ground_truths:
                df = pd.read_csv('%s/%s/%s/%s_%s_%s__metrics.csv' % (results_dir, run_name, opt, opt, ds[0] + rt[0], gt), sep = ';')

                df['reads'] = '%s_%s' % (ds, rt)
                df['gt'] = gt
                df['mode'] = 'swarm'
                df['refinement'] = [m.split('__')[1] for m in df['task']]
                df['ds'] = ds
                df['rt'] = rt

                dfs.append(df)
                    
df_quality = pd.concat(dfs, ignore_index = True)
df_quality.rename(columns = {'task': 'run', 'reads': 'data_set'}, inplace = True)
df_quality.sort_values(by = ['data_set', 'gt', 'tool', 'mode', 'refinement', 'threshold'], inplace = True)

*Column descriptions:*   
`precision`: Quantifies the extent to which amplicons in a cluster are also from the same species.   
`recall`: Measures the proportion of amplicons from the same species that are grouped in the same cluster.   
`adjrandindex`: Measures the agreement between the clusters and the taxonomic assignment and corrects for chance.   

In [7]:
df_quality[['data_set', 'gt', 'tool', 'refinement', 'threshold', 'precision', 'recall', 'adjrandindex']]

,data_set,gt,tool,refinement,threshold,precision,recall,adjrandindex
21,balanced_paired,callahan,swarm,2f,1.0,0.996641,0.952249,0.953953
11,balanced_paired,callahan,swarm,nf,1.0,0.996752,0.951079,0.953145
12,balanced_paired,callahan,swarm,nf,2.0,0.975405,0.954958,0.943879
13,balanced_paired,callahan,swarm,nf,3.0,0.972785,0.969877,0.958978
14,balanced_paired,callahan,swarm,nf,4.0,0.958922,0.971348,0.950872
15,balanced_paired,callahan,swarm,nf,5.0,0.957682,0.971143,0.949760
16,balanced_paired,callahan,swarm,nf,6.0,0.953805,0.977667,0.950825
17,balanced_paired,callahan,swarm,nf,7.0,0.935217,0.977259,0.928314
18,balanced_paired,callahan,swarm,nf,8.0,0.931934,0.974973,0.923524
19,balanced_paired,callahan,swarm,nf,9.0,0.903332,0.977237,0.880799


In [8]:
df_quality.to_csv('%s/df_quality.csv' % eval_dir, sep = ';', index = False)
#df_quality = pd.read_csv('%s/df_quality.csv' % eval_dir, sep = ';')

Combine counting and quality information:

In [9]:
df_c, df_q = df_counts.copy(), df_quality.copy()
drop_cols = ['join_col'] + ['%s_counts' % s for s in set(df_q.columns) & set(df_c.columns)]
df_c['join_col'] = df_c['data_set'] + df_c['tool'] + df_c['mode'] + df_c['refinement'] + df_c['threshold'].apply(str)
df_q['join_col'] = df_q['data_set'] + df_q['tool'] + df_q['mode'] + df_q['refinement'] + df_q['threshold'].apply(str)
df_joined = df_q.join(df_c.set_index('join_col'), on = 'join_col', rsuffix = '_counts').drop(drop_cols, axis = 1)

In [10]:
df_joined.to_csv('%s/df_joined.csv' % eval_dir, sep = ';', index = False)
#df_joined = pd.read_csv('%s/df_joined.csv' % eval_dir, sep = ';')

Determine the maximum, average and N-best average clustering quality (for N = 5).

In [11]:
df_columns = ['data_set', 'gt', 'tool', 'mode', 'refinement', 'precision', 'recall', 'adjrandindex', 'num_input_amplicons', 'input_mass', 'num_clusters', 'num_output_amplicons', 'output_mass', 'ds', 'rt']

max_rows = []
mean_rows = []
nbest_rows = []
n = 5

for (d, g, t, m, f, ds, rt), grp in df_joined.groupby(by = ['data_set', 'gt', 'tool', 'mode', 'refinement', 'ds', 'rt']):
    best = grp.nlargest(1, 'adjrandindex')
    max_rows.append([d, g, t, m, f, best['precision'].values[0], best['recall'].values[0], best['adjrandindex'].values[0], best['num_input_amplicons'].values[0], best['input_mass'].values[0], best['num_clusters'].values[0], best['num_output_amplicons'].values[0], best['output_mass'].values[0], ds, rt])
    mean_rows.append([d, g, t, m, f, grp['precision'].mean(), grp['recall'].mean(), grp['adjrandindex'].mean(), grp['num_input_amplicons'].mean(), grp['input_mass'].mean(), grp['num_clusters'].mean(), grp['num_output_amplicons'].mean(), grp['output_mass'].mean(), ds, rt])
    nbest = grp.nlargest(n, 'adjrandindex')
    nbest_rows.append([d, g, t, m, f, nbest['precision'].mean(), nbest['recall'].mean(), nbest['adjrandindex'].mean(), nbest['num_input_amplicons'].mean(), nbest['input_mass'].mean(), nbest['num_clusters'].mean(), nbest['num_output_amplicons'].mean(), nbest['output_mass'].mean(), ds, rt])
    
df_joined_max = pd.DataFrame(max_rows, columns = df_columns)
df_joined_mean = pd.DataFrame(mean_rows, columns = df_columns)
df_joined_nbest = pd.DataFrame(nbest_rows, columns = df_columns)

In [12]:
df_joined_max.to_csv('%s/df_joined_max.csv' % eval_dir, sep = ';', index = False)
df_joined_mean.to_csv('%s/df_joined_mean.csv' % eval_dir, sep = ';', index = False)
df_joined_nbest.to_csv('%s/df_joined_nbest.csv' % eval_dir, sep = ';', index = False)
#df_joined_max = pd.read_csv('%s/df_joined_max.csv' % eval_dir, sep = ';')
#df_joined_mean = pd.read_csv('%s/df_joined_mean.csv' % eval_dir, sep = ';')
#df_joined_nbest = pd.read_csv('%s/df_joined_nbest.csv' % eval_dir, sep = ';')

In [13]:
df_max = df_joined_max.loc[df_joined_max['gt'] == 'callahan']
df_mean = df_joined_mean.loc[df_joined_mean['gt'] == 'callahan']
df_nbest = df_joined_nbest.loc[df_joined_nbest['gt'] == 'callahan']

For the chosen ground truth, average the maximum, average and N-best average values per data set (e.g. balanced) and read type (e.g. paired).   
Has no effect in this case because there is only one data set per combination of data set and read type .

In [14]:
df_columns = ['data_set', 'gt', 'tool', 'mode', 'refinement', 'precision', 'recall', 'adjrandindex', 'num_input_amplicons', 'input_mass', 'num_clusters', 'num_output_amplicons', 'output_mass', 'ds', 'rt']

def average_complexity(df):
    rows = []
    for (gt, ds, rt, tool, mode, f), grp in df.groupby(by = ['gt', 'ds', 'rt', 'tool', 'mode', 'refinement']):
        rows.append(['%s_%s' % (ds, rt), gt, tool, mode, f, grp['precision'].mean(), grp['recall'].mean(), grp['adjrandindex'].mean(), grp['num_input_amplicons'].mean(), grp['input_mass'].mean(), grp['num_clusters'].mean(), grp['num_output_amplicons'].mean(), grp['output_mass'].mean(), ds, rt])
    return pd.DataFrame(rows, columns = df_columns)

In [15]:
df_joined_max_avg = average_complexity(df_max)
df_joined_mean_avg = average_complexity(df_mean)
df_joined_nbest_avg = average_complexity(df_nbest)

In [16]:
df_joined_max_avg.to_csv('%s/df_joined_max_avg.csv' % eval_dir, sep = ';', index = False)
df_joined_mean_avg.to_csv('%s/df_joined_mean_avg.csv' % eval_dir, sep = ';', index = False)
df_joined_nbest_avg.to_csv('%s/df_joined_nbest_avg.csv' % eval_dir, sep = ';', index = False)
#df_joined_max_avg = pd.read_csv('%s/df_joined_max_avg.csv' % eval_dir, sep = ';')
#df_joined_mean_avg = pd.read_csv('%s/df_joined_mean_avg.csv' % eval_dir, sep = ';')
#df_joined_nbest_avg = pd.read_csv('%s/df_joined_nbest_avg.csv' % eval_dir, sep = ';')

**Maximum clustering quality**

Rank by adjusted Rand index (per data set):

In [17]:
for (d, t), grp in df_joined_max_avg.groupby(by = ['data_set', 'tool']):
    print('Data set: %s / Tool: %s' % (d, t))
    display(grp.sort_values(by = 'adjrandindex', ascending = False))

Data set: balanced_paired / Tool: swarm


,data_set,gt,tool,mode,refinement,precision,recall,adjrandindex,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass,ds,rt
1,balanced_paired,callahan,swarm,swarm,nf,0.972785,0.969877,0.958978,21808.0,467652.0,1747.0,21808.0,467652.0,balanced,paired
0,balanced_paired,callahan,swarm,swarm,2f,0.996641,0.952249,0.953953,21808.0,467652.0,2866.0,21808.0,467652.0,balanced,paired


Data set: balanced_single / Tool: swarm


,data_set,gt,tool,mode,refinement,precision,recall,adjrandindex,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass,ds,rt
2,balanced_single,callahan,swarm,swarm,2f,0.993074,0.948149,0.946025,33523.0,558059.0,5108.0,33523.0,558059.0,balanced,single
3,balanced_single,callahan,swarm,swarm,nf,0.993284,0.943692,0.941401,33523.0,558059.0,8012.0,33523.0,558059.0,balanced,single


Data set: hmp_paired / Tool: swarm


,data_set,gt,tool,mode,refinement,precision,recall,adjrandindex,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass,ds,rt
5,hmp_paired,callahan,swarm,swarm,nf,0.995705,0.728839,0.749133,19882.0,208157.0,2712.0,19882.0,208157.0,hmp,paired
4,hmp_paired,callahan,swarm,swarm,2f,0.998506,0.725966,0.746273,19882.0,208157.0,3143.0,19882.0,208157.0,hmp,paired


Data set: hmp_single / Tool: swarm


,data_set,gt,tool,mode,refinement,precision,recall,adjrandindex,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass,ds,rt
6,hmp_single,callahan,swarm,swarm,2f,0.996923,0.920073,0.935352,73071.0,449409.0,13975.0,73071.0,449409.0,hmp,single
7,hmp_single,callahan,swarm,swarm,nf,0.997724,0.896259,0.909730,73071.0,449409.0,26402.0,73071.0,449409.0,hmp,single


Average the maximum values over all data sets and sort by adjusted Rand index:

In [18]:
rows = []
for (t, m, f), grp in df_joined_max_avg.groupby(by = ['tool', 'mode', 'refinement']):
    rows.append([t, m, f, grp['precision'].mean(), grp['recall'].mean(), grp['adjrandindex'].mean()])
pd.DataFrame(rows, columns = ['tool', 'mode', 'refinement', 'precision', 'recall', 'adjrandindex']).sort_values(by = 'adjrandindex', ascending = False)

,tool,mode,refinement,precision,recall,adjrandindex
0,swarm,swarm,2f,0.996286,0.886609,0.895401
1,swarm,swarm,nf,0.989875,0.884667,0.889811


**Average clustering quality**

Rank by adjusted Rand index (per data set):

In [19]:
for (d, t), grp in df_joined_mean_avg.groupby(by = ['data_set', 'tool']):
    print('Data set: %s / Tool: %s' % (d, t))
    display(grp.sort_values(by = 'adjrandindex', ascending = False))

Data set: balanced_paired / Tool: swarm


,data_set,gt,tool,mode,refinement,precision,recall,adjrandindex,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass,ds,rt
0,balanced_paired,callahan,swarm,swarm,2f,0.996641,0.952249,0.953953,21808.0,467652.0,2866.0,21808.0,467652.0,balanced,paired
1,balanced_paired,callahan,swarm,swarm,nf,0.948790,0.970214,0.931944,21808.0,467652.0,1419.4,21808.0,467652.0,balanced,paired


Data set: balanced_single / Tool: swarm


,data_set,gt,tool,mode,refinement,precision,recall,adjrandindex,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass,ds,rt
2,balanced_single,callahan,swarm,swarm,2f,0.993074,0.948149,0.946025,33523.0,558059.0,5108.0,33523.0,558059.0,balanced,single
3,balanced_single,callahan,swarm,swarm,nf,0.944375,0.962776,0.918179,33523.0,558059.0,2611.9,33523.0,558059.0,balanced,single


Data set: hmp_paired / Tool: swarm


,data_set,gt,tool,mode,refinement,precision,recall,adjrandindex,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass,ds,rt
4,hmp_paired,callahan,swarm,swarm,2f,0.998506,0.725966,0.746273,19882.0,208157.0,3143.0,19882.0,208157.0,hmp,paired
5,hmp_paired,callahan,swarm,swarm,nf,0.952578,0.725148,0.698604,19882.0,208157.0,1659.4,19882.0,208157.0,hmp,paired


Data set: hmp_single / Tool: swarm


,data_set,gt,tool,mode,refinement,precision,recall,adjrandindex,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass,ds,rt
6,hmp_single,callahan,swarm,swarm,2f,0.996923,0.920073,0.935352,73071.0,449409.0,13975.0,73071.0,449409.0,hmp,single
7,hmp_single,callahan,swarm,swarm,nf,0.903524,0.896388,0.751769,73071.0,449409.0,6683.3,73071.0,449409.0,hmp,single


Average the mean values over all data sets and sort by adjusted Rand index:

In [20]:
rows = []
for (t, m, f), grp in df_joined_mean_avg.groupby(by = ['tool', 'mode', 'refinement']):
    rows.append([t, m, f, grp['precision'].mean(), grp['recall'].mean(), grp['adjrandindex'].mean()])
pd.DataFrame(rows, columns = ['tool', 'mode', 'refinement', 'precision', 'recall', 'adjrandindex']).sort_values(by = 'adjrandindex', ascending = False)

,tool,mode,refinement,precision,recall,adjrandindex
0,swarm,swarm,2f,0.996286,0.886609,0.895401
1,swarm,swarm,nf,0.937317,0.888632,0.825124


**N-best average clustering quality**

Rank by adjusted Rand index (per data set):

In [21]:
for (d, t), grp in df_joined_nbest_avg.groupby(by = ['data_set', 'tool']):
    print('Data set: %s / Tool: %s' % (d, t))
    display(grp.sort_values(by = 'adjrandindex', ascending = False))

Data set: balanced_paired / Tool: swarm


,data_set,gt,tool,mode,refinement,precision,recall,adjrandindex,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass,ds,rt
0,balanced_paired,callahan,swarm,swarm,2f,0.996641,0.952249,0.953953,21808.0,467652.0,2866.0,21808.0,467652.0,balanced,paired
1,balanced_paired,callahan,swarm,swarm,nf,0.967989,0.968223,0.952716,21808.0,467652.0,1817.6,21808.0,467652.0,balanced,paired


Data set: balanced_single / Tool: swarm


,data_set,gt,tool,mode,refinement,precision,recall,adjrandindex,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass,ds,rt
2,balanced_single,callahan,swarm,swarm,2f,0.993074,0.948149,0.946025,33523.0,558059.0,5108.0,33523.0,558059.0,balanced,single
3,balanced_single,callahan,swarm,swarm,nf,0.967673,0.956312,0.938298,33523.0,558059.0,4011.4,33523.0,558059.0,balanced,single


Data set: hmp_paired / Tool: swarm


,data_set,gt,tool,mode,refinement,precision,recall,adjrandindex,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass,ds,rt
4,hmp_paired,callahan,swarm,swarm,2f,0.998506,0.725966,0.746273,19882.0,208157.0,3143.0,19882.0,208157.0,hmp,paired
5,hmp_paired,callahan,swarm,swarm,nf,0.982072,0.725281,0.729789,19882.0,208157.0,2447.6,19882.0,208157.0,hmp,paired


Data set: hmp_single / Tool: swarm


,data_set,gt,tool,mode,refinement,precision,recall,adjrandindex,num_input_amplicons,input_mass,num_clusters,num_output_amplicons,output_mass,ds,rt
6,hmp_single,callahan,swarm,swarm,2f,0.996923,0.920073,0.935352,73071.0,449409.0,13975.0,73071.0,449409.0,hmp,single
7,hmp_single,callahan,swarm,swarm,nf,0.951658,0.901155,0.832211,73071.0,449409.0,11330.2,73071.0,449409.0,hmp,single


Average the N-best values over all data sets and sort by adjusted Rand index:

In [22]:
rows = []
for (t, m, f), grp in df_joined_nbest_avg.groupby(by = ['tool', 'mode', 'refinement']):
    rows.append([t, m, f, grp['precision'].mean(), grp['recall'].mean(), grp['adjrandindex'].mean()])
pd.DataFrame(rows, columns = ['tool', 'mode', 'refinement', 'precision', 'recall', 'adjrandindex']).sort_values(by = 'adjrandindex', ascending = False)

,tool,mode,refinement,precision,recall,adjrandindex
0,swarm,swarm,2f,0.996286,0.886609,0.895401
1,swarm,swarm,nf,0.967348,0.887743,0.863253
